# Programming Project #2: Image Quilting

## CS445: Computational Photography - Fall 2022_Xianghe Xu


In [1]:
import cv2
import numpy as np
%matplotlib notebook
import matplotlib
matplotlib.use('Qt5Agg')
from matplotlib import pyplot as plt
import os
import random
import time
import utils

In [2]:
from utils import cut # default cut function for seam finding section

### read images

In [58]:
sample_img_fn = 'samples/bricks_small.jpg' # feel free to change
sample_img = cv2.cvtColor(cv2.imread(sample_img_fn), cv2.COLOR_BGR2RGB)
plt.imshow(sample_img)
plt.show(block=True)

In [4]:
sample_img_fn1 = 'samples/s4.jpg'
sample_img1 = cv2.cvtColor(cv2.imread(sample_img_fn1), cv2.COLOR_BGR2RGB)
plt.imshow(sample_img1)
plt.show(block=True)

In [5]:
sample_img_fn2 = 'samples/s3.jpg'
sample_img2 = cv2.cvtColor(cv2.imread(sample_img_fn2), cv2.COLOR_BGR2RGB)
plt.imshow(sample_img2)
plt.show(block=True)

### Part I: Randomly Sampled Texture (10 pts)

In [6]:
def quilt_random(sample, out_size, patch_size):
    """
    Randomly samples square patches of size patchsize from sample in order to create an output image of size outsize.

    :param sample: numpy.ndarray   The image you read from sample directory
    :param out_size: int            The width of the square output image
    :param patch_size: int          The width of the square sample patch
    :return: numpy.ndarray
    """
    #slice patch
    random.seed(10)
    output = np.zeros((out_size, out_size, 3),np.uint8)
    smaller_output_mul = int(np.floor(out_size/patch_size)) #only uint8 can imshow the picture
    start_location = np.random.randint(0, sample.shape[0] - 1 - patch_size, size = (smaller_output_mul*smaller_output_mul, smaller_output_mul*smaller_output_mul))
    
    order = 0
    for j in range(smaller_output_mul):
        for k in range(smaller_output_mul):
            x_slice = start_location[0, order]
            y_slice = start_location[1, order]
            patch_selected = sample[x_slice: x_slice + patch_size, y_slice: y_slice + patch_size]
            output[j*patch_size:(j + 1)*patch_size, k*patch_size:(k + 1)*patch_size] = patch_selected
            order += 1

    return output



In [7]:
out_size = 200  # change these parameters as needed
patch_size = 15 

res = quilt_random(sample_img, out_size, patch_size)
if res is not None:
    plt.imshow(res)
    plt.show(block=True)


### Part II: Overlapping Patches (30 pts)

In [59]:
def quilt_simple(sample, out_size, patch_size, overlap, tol):
    """
    Randomly samples square patches of size patchsize from sample in order to create an output image of size outsize.
    Feel free to add function parameters
    :param sample: numpy.ndarray
    :param out_size: int
    :param patch_size: int
    :param overlap: int
    :param tol: int
    :return: numpy.ndarray
    """
    def ssd_patch(sample, template, M):
        b,g,r = cv2.split(sample/255.0)
        T_b, T_g, T_r = cv2.split(template)
        
        ssd_b = ((M*T_b)**2).sum() - 2 * cv2.filter2D(b, ddepth=-1, kernel = M*T_b) + cv2.filter2D(b ** 2, ddepth=-1, kernel=M)
        ssd_g = ((M*T_g)**2).sum() - 2 * cv2.filter2D(g, ddepth=-1, kernel = M*T_g) + cv2.filter2D(g ** 2, ddepth=-1, kernel=M)
        ssd_r = ((M*T_r)**2).sum() - 2 * cv2.filter2D(r, ddepth=-1, kernel = M*T_r) + cv2.filter2D(r ** 2, ddepth=-1, kernel=M)
        ssd = ssd_b + ssd_g + ssd_r
        return ssd 

    def choose_sample(ssd, tol, patch_size):
        value = np.zeros((len(ssd) - patch_size, len(ssd) - patch_size))
        half = int(patch_size/2)
        
        for i in range(len(ssd) - patch_size):
            for j in range(len(ssd) - patch_size):
                if i <= half or j <= half:
                    value[i,j] = 1000000
                else:
                    value[i,j] = ssd[i,j]
        value[value == 0] = 1000000
        value_new = value.flatten()
        index = np.argsort(value_new)
        
        if tol == 1:
            choosen = index[0]
        else:
            r = (int)(random()*tol)
            choosen = index[r]
        
        row = int(choosen/(len(ssd) - patch_size))
        col = int(choosen%(len(ssd) - patch_size))
        return row, col
    
    random.seed(10)
    
    # masked templeate: how to track 
    half_patch = int(patch_size/2)
    step = patch_size - overlap
    output = np.zeros((out_size,out_size,3), np.float32)
    # fill the upper-left part
    start_location = np.random.randint(len(sample) - patch_size, size = 2)
    x_start,y_start = start_location[0],start_location[1]
    output[0:patch_size, 0:patch_size] = sample[x_start:(x_start + patch_size), y_start:(y_start + patch_size)]/255.0
    
    i_row = 0
    j_col = step
    while i_row < out_size-patch_size:
        while j_col < out_size-patch_size:
            mask = np.zeros((patch_size,patch_size),np.float32)
            if i_row >= step:
                mask[0:overlap,:] = 1.0
            if j_col >= step:
                mask[:,0:overlap] = 1.0
            template = output[i_row:i_row+patch_size,j_col:j_col+patch_size]
            #ssd making
            ssd = ssd_patch(sample, template, mask)
            ssd[0:1,:] = np.max(ssd)*2
            ssd[:,0:1] = np.max(ssd)*2
            ssd[ssd.shape[0] -1:,:] = np.max(ssd)*2
            ssd[:,ssd.shape[0] -1:] = np.max(ssd)*2
            #find patch
            row, col = choose_sample(ssd,tol,patch_size)
            #paste to output
            output[i_row:i_row+patch_size,j_col:j_col+patch_size] = sample[row-half_patch:row+half_patch+1,col-half_patch:col+half_patch+1]/255.0
            
            j_col += step
        j_col = 0
        i_row += step 
    return output
           

In [60]:
out_size = 240  # change these parameters as needed
patch_size = 35
overlap = 15
tol = 1
res = quilt_simple(sample_img, out_size, patch_size, overlap, tol) #feel free to change parameters to get best results
if res is not None:
    plt.figure(figsize=(10,10))
    plt.imshow(res)
    plt.show(block=True)

### Part III: Seam Finding (20 pts)


In [ ]:
# optional or use cut(err_patch) directly
def customized_cut(bndcost):
    pass

In [63]:
def quilt_cut(sample, out_size, patch_size, overlap, tol):
    """
    Samples square patches of size patchsize from sample using seam finding in order to create an output image of size outsize.
    Feel free to add function parameters
    :param sample: numpy.ndarray
    :param out_size: int
    :param patch_size: int
    :param overlap: int
    :param tol: int
    :return: numpy.ndarray
    """
    
    def ssd_patch(sample, template, M):
        b,g,r = cv2.split(sample/255.0)
        T_b, T_g, T_r = cv2.split(template)
        
        ssd_b = ((M*T_b)**2).sum() - 2 * cv2.filter2D(b, ddepth=-1, kernel = M*T_b) + cv2.filter2D(b ** 2, ddepth=-1, kernel=M)
        ssd_g = ((M*T_g)**2).sum() - 2 * cv2.filter2D(g, ddepth=-1, kernel = M*T_g) + cv2.filter2D(g ** 2, ddepth=-1, kernel=M)
        ssd_r = ((M*T_r)**2).sum() - 2 * cv2.filter2D(r, ddepth=-1, kernel = M*T_r) + cv2.filter2D(r ** 2, ddepth=-1, kernel=M)
        ssd = ssd_b + ssd_g + ssd_r
        return ssd 

    def choose_sample(ssd, tol, patch_size):
        value = np.zeros((len(ssd) - patch_size, len(ssd) - patch_size))
        half = int(patch_size/2)
        
        for i in range(len(ssd) - patch_size):
            for j in range(len(ssd) - patch_size):
                if i <= half or j <= half:
                    value[i,j] = 1000000
                else:
                    value[i,j] = ssd[i,j]
        value[value == 0] = 1000000
        value_new = value.flatten()
        index = np.argsort(value_new)
        
        if tol == 1:
            choosen = index[0]
        else:
            r = (int)(random()*tol)
            choosen = index[r]
        
        row = int(choosen/(len(ssd) - patch_size))
        col = int(choosen%(len(ssd) - patch_size))
        return row, col
    
    # masked templeate: how to track 
    half_patch = int(patch_size/2)
    step = patch_size - overlap
    output = np.zeros((out_size,out_size,3), np.float32)
    # fill the upper-left part
    start_location = np.random.randint(len(sample) - patch_size, size = 2)
    x_start,y_start = start_location[0],start_location[1]
    output[0:patch_size, 0:patch_size] = sample[x_start:(x_start + patch_size), y_start:(y_start + patch_size)]/255.0
    

    i_row = 0
    j_col = step
    while i_row < out_size-patch_size:
        while j_col < out_size-patch_size:
            mask = np.zeros((patch_size,patch_size),np.float32)
            hori_mask = False
            veri_mask = False
            if i_row >= step:
                hori_mask = True
                mask[0:overlap,:] = 1.0
            if j_col >= step:
                veri_mask = True
                mask[:,0:overlap] = 1.0
            template = output[i_row:i_row+patch_size,j_col:j_col+patch_size]
            #ssd making
            ssd = ssd_patch(sample, template, mask)
            ssd[0:1,:] = np.max(ssd)*2
            ssd[:,0:1] = np.max(ssd)*2
            ssd[ssd.shape[0] -1:,:] = np.max(ssd)*2
            ssd[:,ssd.shape[0] -1:] = np.max(ssd)*2
            #find patch
            row, col = choose_sample(ssd,tol,patch_size)
            #paste to output
            patch = sample[row-half_patch:row+half_patch+1,col-half_patch:col+half_patch+1]/255.0
            
            #start seam
            tb, tg, tr = cv2.split(template)
            pb, pg, pr = cv2.split(patch)
            cost = ((tb-pb) ** 2) + ((tg-pg) ** 2) + ((tr-pr) ** 2)
            patch_mask = np.zeros((patch_size,patch_size,3), np.uint8)
            if hori_mask and veri_mask:
                mask1 = np.zeros((patch_size,patch_size,3), np.uint8)
                mask1[:,:,0] = cut(cost.T).T
                mask1[:,:,1] = mask1[:,:,0]
                mask1[:,:,2] = mask1[:,:,0]
                mask2 = np.zeros((patch_size,patch_size,3), np.uint8)
                mask2[:,:,0] = cut(cost)
                mask2[:,:,1] = mask2[:,:,0]
                mask2[:,:,2] = mask2[:,:,0]
                patch_mask = np.bitwise_and(mask1,mask2)
            elif hori_mask:
                patch_mask[:,:,0] = cut(cost)
                patch_mask[:,:,1] = patch_mask[:,:,0]
                patch_mask[:,:,2] = patch_mask[:,:,0]
            elif veri_mask:
                patch_mask[:,:,0] = cut(cost.T).T
                patch_mask[:,:,1] = patch_mask[:,:,0]
                patch_mask[:,:,2] = patch_mask[:,:,0]
            inv_patch_mask = np.where(patch_mask == 0, 1, 0)
            output[i_row:i_row+patch_size,j_col:j_col+patch_size] = patch_mask * patch + template * inv_patch_mask
            
            j_col += step
        j_col = 0
        i_row += step 
    return output

#### show how to draw the second patch

In [51]:
#the version to draw pictures
def quilt_cut(sample, out_size, patch_size, overlap, tol):
    """
    Samples square patches of size patchsize from sample using seam finding in order to create an output image of size outsize.
    Feel free to add function parameters
    :param sample: numpy.ndarray
    :param out_size: int
    :param patch_size: int
    :param overlap: int
    :param tol: int
    :return: numpy.ndarray
    """
    
    def ssd_patch(sample, template, M):
        b,g,r = cv2.split(sample/255.0)
        T_b, T_g, T_r = cv2.split(template)
        
        ssd_b = ((M*T_b)**2).sum() - 2 * cv2.filter2D(b, ddepth=-1, kernel = M*T_b) + cv2.filter2D(b ** 2, ddepth=-1, kernel=M)
        ssd_g = ((M*T_g)**2).sum() - 2 * cv2.filter2D(g, ddepth=-1, kernel = M*T_g) + cv2.filter2D(g ** 2, ddepth=-1, kernel=M)
        ssd_r = ((M*T_r)**2).sum() - 2 * cv2.filter2D(r, ddepth=-1, kernel = M*T_r) + cv2.filter2D(r ** 2, ddepth=-1, kernel=M)
        ssd = ssd_b + ssd_g + ssd_r
        return ssd 

    def choose_sample(ssd,minc,tol,patch_size):
        value = np.zeros((len(ssd) - patch_size, len(ssd) - patch_size))
        half = int(patch_size/2)
        
        for i in range(len(ssd) - patch_size):
            for j in range(len(ssd) - patch_size):
                if i <= half or j <= half:
                    value[i,j] = 1000000
                else:
                    #value[i,j] = np.sum(ssd[i - half:i + half, j - half: j+half])
                    value[i,j] = ssd[i,j]
        value[value == 0] = 1000000
        value_new = value.flatten()
        index = np.argsort(value_new)
        choosen = index[0]
        
        row = int(choosen/(len(ssd) - patch_size))
        col = int(choosen%(len(ssd) - patch_size))
        
        return row, col
    out = np.zeros((out_size,out_size,3), np.float32)
    step = patch_size-overlap
    half = int(patch_size/2)
    small_cost_value = 0.0001
    
    random_ints = np.random.randint(len(sample)-patch_size, size=2)
    xr,yr = random_ints[0],random_ints[1]
    out[0:patch_size,0:patch_size] = sample[xr:xr+patch_size,yr:yr+patch_size]/256.0
    
    patch1 = sample[xr:xr+patch_size,yr:yr+patch_size]
    patch1 = cv2.cvtColor(patch1, cv2.COLOR_BGR2RGB)
    plt.figure()
    plt.title("p1")
    plt.imshow(patch1)
    plt.show(block=True)
   
    
    i = 0
    j = step

    mask = np.zeros((patch_size,patch_size),np.float32)
    h = False
    v = False
    if i >= step:
        h = True
        mask[0:overlap,:] = 1.0
    if j >= step:
        v = True
        mask[:,0:overlap] = 1.0
    template = out[i:i+patch_size,j:j+patch_size]
    ssd = ssd_patch(sample,template,mask)
    minc = np.min(ssd[np.nonzero(ssd)])
    minc = max(minc,small_cost_value)
    row,col = choose_sample(ssd,minc,tol,patch_size)
    patch = sample[row-half:row+half+1,col-half:col+half+1]/256.0
    
    patch2 = sample[row-half:row+half+1,col-half:col+half+1]
    patch2 = cv2.cvtColor(patch2, cv2.COLOR_BGR2RGB)
    plt.figure()
    plt.title("p2")
    plt.imshow(patch2)
    plt.show(block=True)

    
    tb, tg, tr = cv2.split(template)
    pb, pg, pr = cv2.split(patch)
    cost = ((tb-pb) ** 2) + ((tg-pg) ** 2) + ((tr-pr) ** 2)
    patch_mask = np.zeros((patch_size,patch_size,3), np.uint8)
    if h and v:
        mask1 = np.zeros((patch_size,patch_size,3), np.uint8)
        mask1[:,:,0] = cut(cost.T).T
        mask1[:,:,1] = mask1[:,:,0]
        mask1[:,:,2] = mask1[:,:,0]
        mask2 = np.zeros((patch_size,patch_size,3), np.uint8)
        mask2[:,:,0] = cut(cost)
        mask2[:,:,1] = mask2[:,:,0]
        mask2[:,:,2] = mask2[:,:,0]
        patch_mask = np.bitwise_and(mask1,mask2)
    elif h:
        patch_mask[:,:,0] = cut(cost)
        patch_mask[:,:,1] = patch_mask[:,:,0]
        patch_mask[:,:,2] = patch_mask[:,:,0]
    elif v:
        patch_mask[:,:,0] = cut(cost.T).T
        patch_mask[:,:,1] = patch_mask[:,:,0]
        patch_mask[:,:,2] = patch_mask[:,:,0]
    inv_patch_mask = np.where(patch_mask == 0, 1, 0)
    
    plt.figure()
    plt.imshow(patch_mask*256.0)
    plt.show(block=True)
    
    out[i:i+patch_size,j:j+patch_size] = patch_mask*patch + template*inv_patch_mask
    
    plot_overlap = np.zeros((patch_size,2*patch_size-overlap,3), np.float32)
    plot_overlap[:,:,:] = out[i:i+patch_size,j-step:j+patch_size]
    
    plot_mask = np.zeros((patch_size,patch_size,3), np.uint8)
    for ii in range(patch_size):
        for jj in range(1,patch_size):
            if patch_mask[ii,jj,0] == patch_mask[ii,jj-1,0]:
                pass
            else:
                plot_mask[ii,jj,0] = 0
                plot_mask[ii,jj,1] = 0
                plot_mask[ii,jj,2] = 1
                
    plt.figure()
    plot_overlap_out = cv2.cvtColor(plot_overlap, cv2.COLOR_BGR2RGB)
    plt.imshow(plot_overlap_out)
    plt.show(block=True)
    
    plot_overlap[0:patch_size,0:patch_size,:] += plot_mask
    plot_overlap = cv2.cvtColor(plot_overlap, cv2.COLOR_BGR2RGB)
    plt.figure()
    plt.imshow(plot_overlap)
    plt.show(block=True)
    
    plt.figure()
    plt.imshow(cost[0:patch_size,0:overlap])
    
    for ii in range(patch_size):
        for jj in range(overlap):
            if plot_mask[ii,jj,2]:
                cost[ii,jj] = 1.0
    plt.figure()
    plt.imshow(cost[0:patch_size,0:overlap])
    plt.show(block=True)
    
    return out

In [64]:
out_size = 240  # change these parameters as needed
patch_size = 35
overlap = 15
tol = 1
res = quilt_cut(sample_img, out_size, patch_size, overlap, tol)
if res is not None:
    plt.figure(figsize=(15,15))
    plt.imshow(res)
    plt.show(block=True)

### part IV: Texture Transfer (30 pts)

In [8]:
def texture_transfer(sample, patch_size, overlap, tol, target, alpha):
    """
    Feel free to add function parameters
    """
    def ssd_patch(sample, template, M):
        b,g,r = cv2.split(sample/255.0)
        T_b, T_g, T_r = cv2.split(template)
        
        ssd_b = ((M*T_b)**2).sum() - 2 * cv2.filter2D(b, ddepth=-1, kernel = M*T_b) + cv2.filter2D(b ** 2, ddepth=-1, kernel=M)
        ssd_g = ((M*T_g)**2).sum() - 2 * cv2.filter2D(g, ddepth=-1, kernel = M*T_g) + cv2.filter2D(g ** 2, ddepth=-1, kernel=M)
        ssd_r = ((M*T_r)**2).sum() - 2 * cv2.filter2D(r, ddepth=-1, kernel = M*T_r) + cv2.filter2D(r ** 2, ddepth=-1, kernel=M)
        ssd = ssd_b + ssd_g + ssd_r
        return ssd 

    def choose_sample(ssd, tol, patch_size):
        value = np.zeros((ssd.shape[0] - patch_size, ssd.shape[1] - patch_size))
        half = int(patch_size/2)
        
        for i in range(ssd.shape[0] - patch_size):
            for j in range(ssd.shape[1] - patch_size):
                if i <= half or j <= half:
                    value[i,j] = 1000000
                else:
                    value[i,j] = ssd[i,j]
        value[value == 0] = 1000000
        value_new = value.flatten()
        index = np.argsort(value_new)
        
        if tol == 1:
            choosen = index[0]
        else:
            r = (int)(random()*tol)
            choosen = index[r]
        
        row = int(choosen/(ssd.shape[1] - patch_size))
        col = int(choosen%(ssd.shape[1] - patch_size))
        return row, col
    
    width = len(target[0,:,:])
    height = len(target[:,0,:])
    
    # masked templeate: how to track 
    half_patch = int(patch_size/2)
    step = patch_size - overlap
    output = np.zeros((height,width,3), np.float32)
    # fill the upper-left part
    start_location = np.random.randint(len(sample) - patch_size, size = 2)
    x_start,y_start = start_location[0],start_location[1]
    output[0:patch_size, 0:patch_size] = sample[x_start:(x_start + patch_size), y_start:(y_start + patch_size)]/255.0
    
    
    i_row = 0
    j_col = step
    while i_row < height-patch_size:
        while j_col < width-patch_size:
            mask = np.zeros((patch_size,patch_size),np.float32)
            hori_mask = False
            veri_mask = False
            if i_row >= step:
                hori_mask = True
                mask[0:overlap,:] = 1.0
            if j_col >= step:
                veri_mask = True
                mask[:,0:overlap] = 1.0
                
            #slice patches
            template = output[i_row:i_row+patch_size,j_col:j_col+patch_size]
            transfer = target[i_row:i_row+patch_size,j_col:j_col+patch_size]/255.0
            #ssd making
            ssd_p = ssd_patch(sample,template,mask)
            ssd_t = ssd_patch(sample,transfer,mask)
            ssd = alpha*ssd_p+(1-alpha)*ssd_t
            ssd[0:1,:] = np.max(ssd)*2
            ssd[:,0:1] = np.max(ssd)*2
            ssd[ssd.shape[0] -1:,:] = np.max(ssd)*2
            ssd[:,ssd.shape[0] -1:] = np.max(ssd)*2
            #find patch
            row,col = choose_sample(ssd, tol,patch_size)
            #paste to output
            patch = sample[row-half_patch:row+half_patch+1,col-half_patch:col+half_patch+1]/255.0
          
            
            tb, tg, tr = cv2.split(template)
            pb, pg, pr = cv2.split(patch)
            cost = ((tb-pb) ** 2) + ((tg-pg) ** 2) + ((tr-pr) ** 2)
            patch_mask = np.zeros((patch_size,patch_size,3), np.uint8)
            if hori_mask and veri_mask:
                mask1 = np.zeros((patch_size,patch_size,3), np.uint8)
                mask1[:,:,0] = cut(cost.T).T
                mask1[:,:,1] = mask1[:,:,0]
                mask1[:,:,2] = mask1[:,:,0]
                mask2 = np.zeros((patch_size,patch_size,3), np.uint8)
                mask2[:,:,0] = cut(cost)
                mask2[:,:,1] = mask2[:,:,0]
                mask2[:,:,2] = mask2[:,:,0]
                patch_mask = np.bitwise_and(mask1,mask2)
            elif hori_mask:
                patch_mask[:,:,0] = cut(cost)
                patch_mask[:,:,1] = patch_mask[:,:,0]
                patch_mask[:,:,2] = patch_mask[:,:,0]
            elif veri_mask:
                patch_mask[:,:,0] = cut(cost.T).T
                patch_mask[:,:,1] = patch_mask[:,:,0]
                patch_mask[:,:,2] = patch_mask[:,:,0]
                
            patch_mask_inverse = np.where(patch_mask == 0, 1, 0)
            output[i_row:i_row+patch_size,j_col:j_col+patch_size] = patch_mask*patch + template*patch_mask_inverse
            j_col += step
        j_col = 0
        i_row += step 
    return output

In [9]:
sample_img_dir = 'samples/toast.jpg' # feel free to change
sample_img = None
if os.path.exists(sample_img_dir):
    sample_img = cv2.imread(sample_img_dir)
    sample_out = cv2.cvtColor(sample_img, cv2.COLOR_BGR2RGB)
    plt.figure()
    plt.imshow(sample_out)
    plt.show(block=True)

target_img_dir = 'samples/feynman.tiff' # feel free to change
target_img = None
if os.path.exists(target_img_dir):
    target_img = cv2.imread(target_img_dir)
    target_out = cv2.cvtColor(target_img, cv2.COLOR_BGR2RGB)
    plt.figure()
    plt.imshow(target_out)
    plt.show(block=True)

In [10]:
# load/process appropriate input texture and guidance images

patch_size = 35
overlap = 15
tol = 1
alpha = 0.1
res = texture_transfer(sample_out, patch_size, overlap, tol, target_out, alpha)

plt.figure(figsize=(10,10))
plt.imshow(res)
plt.show()
plt.show(block=True)

### Bells & Whistles

(10 pts) Create and use your own version of cut.m. To get these points, you should create your own implementation without basing it directly on the provided function (you're on the honor code for this one). 

You can simply copy your customized_cut(bndcost) into the box below so that it is easier for us to grade

(15 pts) Implement the iterative texture transfer method described in the paper. Compare to the non-iterative method for two examples.

(up to 20 pts) Use a combination of texture transfer and blending to create a face-in-toast image like the one on top. To get full points, you must use some type of blending, such as feathering or Laplacian pyramid blending.

(up to 40 pts) Extend your method to fill holes of arbitrary shape for image completion. In this case, patches are drawn from other parts of the target image. For the full 40 pts, you should implement a smart priority function (e.g., similar to Criminisi et al.).